# Assignment 3.1. Sequence Classification

<b>Group 58</b>
* <b> Student 1 </b> : Luc Reinink, 1068948
* <b> Student 2 </b> : Gerrit Merz, 1553410

# Task: Aspect-level Sentiment Classification(10pt)

Reading material:
- [1] R. He, WS. Lee & D. Dahlmeier. Exploiting document knowledge for aspect-level sentiment classification. 2018. https://arxiv.org/abs/1806.04346.


Build an attention-based aspect-level sentiment classification model with biLSTM. Your model shall include:

- BiLSTM network that learns sentence representation from input sequences.
- Attention network that assigns attention score over a sequence of biLSTM hidden states based on aspect terms representation.
- Fully connected network that predicts sentiment label, given the representation weighted by the attention score.

Requirements:

- You shall train your model bsaed on transferring learning. That is, you need first train your model on documnet-level examples. Then the learned weights will be used to initialize aspect-level model and fine tune it on aspect-level examples.
- You shall use the alignment score function in attention network as following expression:$$f_{score}(h,t)=tanh(h^TW_a t)$$
- You shall evaluate the trained model on the provided test set and show the accuracy on test set.


In [0]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [0]:
import _pickle as cPickle


#Load Data

In [0]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [0]:
aspect_path = "/content/drive/My Drive/2IMM10 Deep Learning/Practicals/Practical 5/Data/aspect-level"


vocab = read_pickle(aspect_path, 'all_vocab.pkl')

train_x = read_pickle(aspect_path, 'train_x.pkl')
train_y = read_pickle(aspect_path, 'train_y.pkl')
dev_x = read_pickle(aspect_path, 'dev_x.pkl')
dev_y = read_pickle(aspect_path, 'dev_y.pkl')
test_x = read_pickle(aspect_path, 'test_x.pkl')
test_y = read_pickle(aspect_path, 'test_y.pkl')

train_aspect = read_pickle(aspect_path, 'train_aspect.pkl')
dev_aspect = read_pickle(aspect_path, 'dev_aspect.pkl')
test_aspect = read_pickle(aspect_path, 'test_aspect.pkl')


pretrain_data = read_pickle(aspect_path, 'pretrain_data.pkl')
pretrain_label = read_pickle(aspect_path, 'pretrain_label.pkl')

In [0]:
class Dataiterator_doc():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y
class Dataiterator_aspect():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, aspect_data, seq_length=32, decoder_dim=300, batch_size=32):
        
        len_aspect_data = len(aspect_data[0])
        #self.len_doc_data = len(doc_data[0])
        
        self.X_aspect = aspect_data[0] 
        self.y_aspect = aspect_data[1]
        self.aspect_terms = aspect_data[2]  
        self.num_data = len_aspect_data
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X_aspect = self.X_aspect[np.array(X_ids)] # X values (encoder input) per batch
        batch_y_aspect = self.y_aspect[np.array(X_ids)] # y_in values (decoder input) per batch
        batch_aspect_terms = self.aspect_terms[np.array(X_ids)]
        
        return batch_X_aspect, batch_y_aspect, batch_aspect_terms

          
    def all(self): # return all data examples
        return self.X_aspect, self.y_aspect, self.aspect_terms

In [0]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Lambda, Dropout, LSTM,Bidirectional
from keras.layers import Reshape, Activation, RepeatVector, concatenate, Concatenate, Dot, Multiply
import keras.backend as K
from keras.engine.topology import Layer
from keras import initializers
from keras import regularizers
from keras import constraints


Using TensorFlow backend.


In [0]:
overal_maxlen = 82
overal_maxlen_aspect = 7


#Define Attention Network Layer
- Define class for Attention Layer
- You need to finish the code for calculating the attention weights

In [0]:
class Attention(Layer):
    def __init__(self,  **kwargs):
        """
        Keras Layer that implements an Content Attention mechanism.
        Supports Masking.
        """
       
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert type(input_shape) == list
       
        self.steps = input_shape[0][1]

        self.W = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),)

        self.built = True

    def compute_mask(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        assert type(mask) == list
        return None

    def call(self, input_tensor, mask=None):
        x = input_tensor[0]
        aspect = input_tensor[1]

        aspect = K.transpose(K.dot(self.W, K.transpose(aspect)))
        aspect = K.expand_dims(aspect, axis=-2)
        aspect = K.repeat_elements(aspect, self.steps, axis=1)
        eij = K.sum(x*aspect, axis=-1)

        eij = K.tanh(eij)

        a = K.exp(eij)

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        return a

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1])

In [0]:
class Average(Layer):
  
    def __init__(self, mask_zero=True, **kwargs):
        self.mask_zero = mask_zero
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x,mask=None):
        if self.mask_zero:           
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
            return K.sum(x, axis=1) / (K.sum(mask, axis=1) + K.epsilon())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def compute_mask(self, x, mask):
        return None

#Establish computation Grah for model
- Input tensors
- Shared WordEmbedding layer 
- Attention network layer  
- Shared BiLSTM layer
- Shared fully connected layer(prediction layer)


In [0]:
dropout = 0.5     
recurrent_dropout = 0.1
vocab_size = len(vocab)
num_outputs = 3 # labels

##Input tensors

In [0]:
#YOUR CODE HERE ##### Inputs #####
sentence_input = Input(shape=(overal_maxlen,), dtype='int32', 
                       name='sentence_input')
aspect_input = Input(shape=(overal_maxlen_aspect,), dtype='int32', 
                     name='aspect_input')
pretrain_input = Input(shape=(None,), dtype='int32', name='pretrain_input')

##Shared WordEmbedding layer

In [0]:
#YOUR CODE HERE### represent aspect as averaged word embedding ###
# We take the embedding layer's dimensions from the paper.
emb_dim = 300

word_emb = Embedding(vocab_size, emb_dim, mask_zero=True, name='word_emb')
aspect_term_embs = word_emb(aspect_input)
aspect_embs = Average(mask_zero=True, name='aspect_embs')(aspect_term_embs)

In [0]:
#YOUR CODE HERE ### sentence representation from embedding ###
sentence_embs = word_emb(sentence_input)
pretrain_embs = word_emb(pretrain_input)

##Shared BiLSTM layer

In [0]:
#YOUR CODE HERE ### sentence representation from embedding ###
# We take the dimension of LSTM hidden vectors from the paper.
lstm_dim = 300 

bilstm = Bidirectional(LSTM(lstm_dim, return_sequences=True, dropout=dropout, 
                            recurrent_dropout=recurrent_dropout, name='BiLSTM'))

In [0]:
sentence_bilstm = bilstm(sentence_embs)
pretrain_bilstm = bilstm(pretrain_embs)
print(sentence_bilstm.shape)

(None, 82, 600)


##Attention Layer

In [0]:
##YOUR CODE HERE
att_weights = Attention(name='attention_layer')([sentence_bilstm, 
                                                     aspect_embs])

##Prediction Layer

In [0]:
class WeightedSum(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(WeightedSum, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        assert type(mask) == list

        x = input_tensor[0]
        a = input_tensor[1]

        a = K.expand_dims(a)
        weighted_input = x * a

        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][-1])

    def compute_mask(self, x, mask=None):
        return None

In [0]:
#YOUR CODE HERE
prediction_layer = Dense(num_outputs, activation="softmax")
aspect_probs = prediction_layer(WeightedSum()([sentence_bilstm, att_weights]))

pretrain_avg = Average(mask_zero=True)(pretrain_bilstm)
doc_probs = prediction_layer(pretrain_avg)

#Build Models for document-level and aspect-level data
- The two models shared the embedding, BiLSTM, Prediction Layer

In [0]:
### YOUR CODE HERE
model1 = Model(inputs=[pretrain_input], outputs=[doc_probs])
model2 = Model(inputs=[sentence_input, aspect_input], outputs=[aspect_probs])

In [0]:
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pretrain_input (InputLayer)  (None, None)              0         
_________________________________________________________________
word_emb (Embedding)         multiple                  3000900   
_________________________________________________________________
bidirectional_1 (Bidirection multiple                  1442400   
_________________________________________________________________
average_1 (Average)          (None, 600)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1803      
Total params: 4,445,103
Trainable params: 4,445,103
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_input (InputLayer)     (None, 82)           0                                            
__________________________________________________________________________________________________
aspect_input (InputLayer)       (None, 7)            0                                            
__________________________________________________________________________________________________
word_emb (Embedding)            multiple             3000900     aspect_input[0][0]               
                                                                 sentence_input[0][0]             
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) multiple             1442400     word_emb[1][0]             

#Train Model
- First Train model on document-level data.
- Then Train  model on aspect-level data

##Train on document-level data

In [0]:
import keras.optimizers as opt
optimizer=opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
batch_size = 128
train_steps_epoch = len(pretrain_data)/batch_size
batch_train_iter_doc = Dataiterator_doc(pretrain_data, pretrain_label, batch_size)

In [0]:
###YOUR CODE HERE###
epochs = 4 # More than 5 crashes
model1.fit_generator(batch_train_iter_doc, epochs=epochs, 
                             steps_per_epoch=train_steps_epoch)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
235/234 [==============================] - 336s 1s/step - loss: 0.9429 - categorical_accuracy: 0.5430
Epoch 2/4
235/234 [==============================] - 337s 1s/step - loss: 0.7808 - categorical_accuracy: 0.6547
Epoch 3/4
235/234 [==============================] - 341s 1s/step - loss: 0.7491 - categorical_accuracy: 0.6673
Epoch 4/4
235/234 [==============================] - 345s 1s/step - loss: 0.7239 - categorical_accuracy: 0.6739


##Train on aspect-level data

In [0]:
train_steps_epoch = len(train_x)/batch_size
batch_train_iter_aspect = Dataiterator_aspect([train_x, train_y, train_aspect], batch_size)
val_steps_epoch = len(dev_x)/batch_size
batch_val_iter_aspect = Dataiterator_aspect([dev_x, dev_y, dev_aspect], batch_size)

import keras.optimizers as opt
optimizer = opt.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=10, clipvalue=0)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [0]:
def train_gen():
    while True:
        train_batches = [[[X, aspect], [y]] for X, y, \
                            aspect in batch_val_iter_aspect]
        for train_batch in train_batches:
            yield train_batch

In [0]:
### YOUR CODE HERE
epochs = 10
model2.fit_generator(train_gen(), epochs=epochs, 
                             steps_per_epoch=train_steps_epoch)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15/14 [===============================] - 7s 467ms/step - loss: 1.0060 - categorical_accuracy: 0.5604
Epoch 2/10
15/14 [===============================] - 6s 416ms/step - loss: 0.9450 - categorical_accuracy: 0.6000
Epoch 3/10
15/14 [===============================] - 6s 411ms/step - loss: 0.9462 - categorical_accuracy: 0.6146
Epoch 4/10
15/14 [===============================] - 6s 411ms/step - loss: 0.8295 - categorical_accuracy: 0.6562
Epoch 5/10
15/14 [===============================] - 6s 411ms/step - loss: 0.8811 - categorical_accuracy: 0.6292
Epoch 6/10
15/14 [===============================] - 6s 417ms/step - loss: 0.7486 - categorical_accuracy: 0.6896
Epoch 7/10
15/14 [===============================] - 6s 410ms/step - loss: 0.8460 - categorical_accuracy: 0.6625
Epoch 8/10
15/14 [===============================] - 6s 416ms/step - loss: 0.7159 - categorical_accuracy: 0.7083
Epoch 9/10
15/14 [===============================] - 6s 414ms/step - loss: 0.7276 - categorical_

##Evaluating on test set
- show the accuracy

In [0]:
##YOUR CODE HERE
loss, acc = model1.evaluate(test_x, test_y)
print("Model 1 accuracy =", acc * 100)

638/638 [==============================] - 1s 2ms/step
Model 1 accuracy = 59.71786975860596


In [0]:
loss, acc = model2.evaluate([test_x, test_aspect], test_y)
print("Model 2 accuracy =", acc * 100)

638/638 [==============================] - 1s 2ms/step
Model 2 accuracy = 59.874606132507324
